# RecursiveCharacterTextSplitter 완전 가이드

## 개요

**RecursiveCharacterTextSplitter**는 LangChain에서 텍스트를 의미 단위로 지능적으로 분할하는 텍스트 분할 도구입니다.

### RecursiveCharacterTextSplitter의 특징

일반적인 텍스트 분할기와 달리 이 도구는 자연스러운 문서 구조를 따라 텍스트를 분할합니다:

| 분할 우선순위 | 설명 | 목적 |
|-------------|------|------|
| 단락 우선 | 의미가 완성된 단락을 최대한 유지 | 문맥 보존 |
| 문장 단위 | 단락이 크면 문장 단위로 분할 | 의미 단위 유지 |
| 단어 단위 | 문장도 크면 단어 단위로 분할 | 최소 단위 분할 |
| 재귀적 접근 | 크기가 맞을 때까지 반복 분할 | 정확한 크기 제어 |

### 동작 원리

RecursiveCharacterTextSplitter는 우선순위가 있는 분할 전략을 사용합니다:

1. **구분자 목록**: `["\n\n", "\n", " ", ""]` 순서로 시도
2. **재귀적 분할**: 청크가 목표 크기에 맞을 때까지 반복
3. **의미 보존**: 가능한 한 관련된 내용을 함께 유지

### 분할 과정 예시

```
전체 텍스트
├── 단락별 분할 (\n\n 구분자) - 1차 시도
├── 문장별 분할 (\n 구분자) - 청크가 크면
├── 단어별 분할 (" " 구분자) - 여전히 크면
└── 문자별 분할 ("") - 최후 수단
```

### 학습 목표

본 튜토리얼에서는 다음 내용을 다룹니다:

1. **파일 읽기** - 실제 텍스트 데이터 준비
2. **분할기 설정** - chunk_size, chunk_overlap 등 핵심 매개변수 이해
3. **텍스트 분할** - 청크로 나누는 실전 예제
4. **결과 분석** - 분할된 결과 확인 및 최적화

### 핵심 매개변수

| 매개변수 | 설명 | 기본값 |
|----------|------|--------|
| chunk_size | 각 청크의 최대 문자 수 | 4000 |
| chunk_overlap | 청크 간 중복되는 문자 수 | 200 |
| separators | 분할에 사용할 구분자 목록 | ["\n\n", "\n", " ", ""] |
| length_function | 길이 측정 함수 | len |

## 환경 설정

RecursiveCharacterTextSplitter를 사용하기 전에 필요한 환경을 설정합니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

---

# Part 1: 데이터 준비

## 1.1 샘플 텍스트 파일 읽기

실제 텍스트 분할을 실습하기 위해 샘플 데이터를 준비합니다.

### 샘플 파일 정보

| 항목 | 내용 |
|------|------|
| 파일명 | `appendix-keywords.txt` |
| 용도 | RecursiveCharacterTextSplitter 테스트용 샘플 텍스트 |
| 특징 | 다양한 단락과 문장이 포함된 실제 문서 |

### Python으로 텍스트 파일 읽기

Python의 **with open()** 구문을 사용하여 텍스트 파일을 안전하게 읽어옵니다.

#### with open() 구문의 장점

| 장점 | 설명 |
|------|------|
| 자동 파일 닫기 | 파일을 사용한 후 자동으로 닫아줌 |
| 안전한 파일 처리 | 오류가 발생해도 파일이 안전하게 닫힘 |
| 깔끔한 코드 | 파일 열기/닫기를 별도로 관리할 필요 없음 |

In [ ]:
# 샘플 텍스트 파일을 열어서 읽기
with open("./data/appendix-keywords.txt") as f:
    file = f.read()  # 파일 전체 내용을 문자열로 읽어서 file 변수에 저장

### 파일 내용 미리보기

읽어온 텍스트의 내용을 **처음 500자**만 출력하여 확인합니다.

#### 일부 출력하는 이유

| 이유 | 설명 |
|------|------|
| 효율적인 확인 | 전체 내용은 길어서 가독성이 떨어짐 |
| 구조 파악 | 텍스트의 전반적인 구조와 패턴 파악 |
| 빠른 검증 | 파일이 정상적으로 읽혔는지 신속히 확인 |

In [ ]:
# 파일 내용의 처음 500자만 출력하여 미리보기
print(file[:500])

---

# Part 2: RecursiveCharacterTextSplitter 심화 학습

## 2.1 라이브러리 import

LangChain의 **text_splitters** 모듈에서 RecursiveCharacterTextSplitter를 가져옵니다.

### langchain_text_splitters 모듈

LangChain에서 제공하는 다양한 텍스트 분할기들의 집합입니다:

| 분할기 | 특징 | 용도 |
|--------|------|------|
| RecursiveCharacterTextSplitter | 가장 일반적이고 지능적인 분할기 | 대부분의 텍스트 분할 작업 |
| TokenTextSplitter | 토큰 단위로 분할 (GPT 토큰 기준) | 토큰 제한이 중요한 경우 |
| CharacterTextSplitter | 단순 문자 기준 분할 | 단순한 분할이 필요한 경우 |
| MarkdownHeaderTextSplitter | 마크다운 헤더 기준 분할 | 마크다운 문서 처리 |

In [ ]:
# LangChain 텍스트 분할기 라이브러리에서 RecursiveCharacterTextSplitter 가져오기
from langchain_text_splitters import RecursiveCharacterTextSplitter

## 2.2 분할기 설정하기

**RecursiveCharacterTextSplitter** 인스턴스를 생성하고 핵심 매개변수들을 설정합니다.

### 핵심 매개변수 상세 설명

#### chunk_size: 250
- **의미**: 각 청크(조각)의 최대 문자 수
- **예시**: "안녕하세요"는 5자, "Hello World"는 11자
- **주의사항**: 실제 크기는 구분자 위치에 따라 약간 달라질 수 있음

#### chunk_overlap: 50
- **의미**: 인접한 청크 간 중복되는 문자 수
- **목적**: 문맥이 끊어지는 것을 방지
- **예시**: 청크1의 마지막 50자가 청크2의 첫 50자와 동일

#### length_function: len
- **의미**: 텍스트 길이를 측정하는 함수
- **기본값**: Python의 `len()` 함수 (문자 수 기준)
- **대안**: 토큰 수 기준 함수도 사용 가능

#### is_separator_regex: False
- **의미**: 구분자를 정규식으로 해석할지 여부
- **False**: 구분자를 일반 문자열로 처리 (권장)
- **True**: 구분자를 정규식 패턴으로 처리 (고급 사용자용)

### 매개변수 비유

책장 정리 작업에 비유하면 다음과 같습니다:

| 매개변수 | 비유 |
|----------|------|
| chunk_size | 각 책장 칸의 최대 수납 공간 |
| chunk_overlap | 책들이 약간씩 겹쳐지도록 배치 (연결성 유지) |
| length_function | 책의 두께를 재는 방법 (페이지 수 또는 무게) |
| is_separator_regex | 책 분류 기준의 복잡성 정도 |

In [ ]:
# RecursiveCharacterTextSplitter 인스턴스 생성 및 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=50,
    separators=[r"\n---\n", "\n\n", "\n", " ", ""],
    is_separator_regex=True,
)

# 텍스트를 청크로 분할
chunks = text_splitter.split_text(file)

for i, chunk in enumerate(chunks, 1):
    print(f"[청크 {i}]")
    print(chunk)
    print("=" * 60)

## 2.3 실제 텍스트 분할하기

설정한 분할기로 실제 텍스트를 **Document 객체**로 분할합니다.

### create_documents() vs split_text() 비교

| 구분 | create_documents() | split_text() |
|------|-------------------|-------------|
| 반환 타입 | List[Document] | List[str] |
| 포함 내용 | 텍스트 + 메타데이터 | 텍스트만 |
| 장점 | 벡터 스토어/체인에서 바로 사용 가능 | 간단하고 직관적 |
| 사용 시기 | 실제 RAG 시스템 구축 | 단순 텍스트 분할 테스트 |

### Document 객체 구조

LangChain의 **Document**는 텍스트와 메타데이터를 함께 저장하는 컨테이너입니다:

```python
Document(
    page_content="실제 텍스트 내용",  # 분할된 텍스트
    metadata={"source": "파일명", "chunk_id": 0}  # 부가 정보
)
```

In [ ]:
# create_documents()로 텍스트를 Document 객체들로 분할
texts = text_splitter.create_documents([file])

# 분할된 첫 번째 Document 출력 (내용과 메타데이터 모두 포함)
print(texts[0])
print("===" * 20)  # 구분선
# 분할된 두 번째 Document 출력
print(texts[1])

## 2.4 순수 텍스트 분할 - split_text()

**split_text()** 메서드를 사용하여 순수한 문자열 리스트로 분할합니다.

### split_text() 특징

#### 장점
- **빠른 처리**: Document 객체 생성 과정이 없어 처리 속도가 빠름
- **직관적**: 단순히 문자열 리스트로 반환
- **가벼움**: 메타데이터 없이 텍스트만 처리

#### 단점
- **추적 어려움**: 어느 청크가 원본의 어느 부분인지 파악하기 어려움
- **연결 부족**: LangChain 체인에서 바로 사용하기 어려움
- **메타데이터 없음**: 부가 정보 없이 텍스트만 제공

### 사용 상황별 메서드 선택

| 상황 | 추천 메서드 | 이유 |
|------|-------------|------|
| RAG 시스템 구축 | `create_documents()` | 벡터 스토어 저장 시 메타데이터 필요 |
| 간단한 테스트 | `split_text()` | 빠르고 직관적인 결과 확인 |
| 텍스트 분석 | `split_text()` | 순수 문자열로 분석 작업 진행 |
| 프로덕션 환경 | `create_documents()` | 완전한 기능과 추적성 필요 |

In [ ]:
# split_text()로 순수 문자열 리스트로 분할하고 처음 2개 청크만 출력
text_splitter.split_text(file)[:2]

---

# 정리 및 핵심 포인트

## 학습 내용 요약

**RecursiveCharacterTextSplitter**의 핵심 기능을 학습했습니다.

### 학습한 내용

1. **데이터 준비**: 실제 텍스트 파일을 안전하게 읽어오기
2. **분할기 설정**: 핵심 매개변수들의 의미와 활용법
3. **Document 분할**: `create_documents()`로 메타데이터와 함께 분할
4. **텍스트 분할**: `split_text()`로 순수 문자열 리스트로 분할

### 핵심 매개변수 복습

| 매개변수 | 설명 |
|----------|------|
| chunk_size | 각 청크의 최대 문자 수 |
| chunk_overlap | 청크 간 중복 문자 수 (맥락 유지용) |
| length_function | 길이 측정 함수 (기본: `len`) |
| is_separator_regex | 구분자 정규식 사용 여부 |

### 다음 단계 활용

RecursiveCharacterTextSplitter를 활용한 다음 단계:

- **벡터 스토어 구축**: 분할된 텍스트를 임베딩으로 변환
- **RAG 시스템 구축**: 질문-답변 시스템 개발
- **텍스트 분석**: 대용량 문서 분석 및 처리
- **성능 최적화**: chunk_size와 overlap 조정으로 성능 개선

### 실무 권장 설정

| 텍스트 유형 | chunk_size | chunk_overlap | 설명 |
|-------------|------------|---------------|------|
| 일반 텍스트 | 1000 | 200 | 일반적인 문서에 적합 |
| 대화형 텍스트 | 500 | 50 | 짧은 대화나 메시지 |
| 기술 문서 | 1500 | 300 | 복잡한 기술 내용 |
| 테스트용 | 250 | 50 | 본 튜토리얼 설정 |

### 결론

RecursiveCharacterTextSplitter의 기본 개념과 사용법을 익혔습니다. 이제 본격적인 RAG 시스템 구축을 위한 기반이 마련되었습니다.